# Inference On Soccernet Challenge Dataset (EasyOCR)

## Introduction

This notebook aims to demonstrate the process of performing inference on the Soccernet Challenge dataset using EasyOCR, a popular Python library for optical character recognition (OCR). The Soccernet Challenge dataset consists of images containing players frame extracted from soccer match footage.

## Notebook Overview

In this notebook, we will cover the following steps:

1. **Loading the Pretrained EasyOCR Model:** We will load a pretrained EasyOCR model capable of recognizing text from images. EasyOCR provides robust and efficient OCR capabilities, making it suitable for various text recognition tasks.

2. **Performing Inference:** Using the pretrained EasyOCR model, we will perform inference on the images from the Soccernet Challenge dataset to extract jersey numbers. We will demonstrate how to process individual images and batch process multiple images efficiently.

In this notebook, I'll be working on performing optical character recognition (OCR) on images containing soccer players. To accomplish this task, I'll be using the EasyOCR library, a powerful Python package for text detection and recognition in images. EasyOCR offers pretrained models and a simple API for performing OCR tasks with minimal setup and configuration.

Additionally, I'll install the Ultralytics library, which provides utilities and tools for working with computer vision models, including object detection, image classification, and more. While not directly used in this notebook, Ultralytics may offer additional functionalities and insights for future experiments or analyses.

### Package Installation

Before I proceed, let's install the necessary packages:
- **EasyOCR**: For performing optical character recognition on images.
- **Ultralytics**: Providing utilities and tools for computer vision tasks.

In this notebook, I'll implement a solution for extracting jersey numbers from soccer players' frames using the YOLOv5 Pose model provided by Ultralytics. The YOLOv5 Pose model is a variant of the YOLO (You Only Look Once) object detection architecture specifically designed for detecting human poses and keypoints. By leveraging this model along with custom logic, I aim to accurately localize and extract jersey numbers from player frames in soccer match footage.

Let's install these packages before moving forward.

In [ ]:
!pip install easyocr
!pip install ultralytics

### Imports

Before we begin coding, let's import the necessary libraries and modules that we'll be using throughout the notebook. These include popular libraries such as Pandas, TensorFlow, NumPy, Matplotlib, OpenCV, and scikit-learn, along with specific modules and classes from TensorFlow's Keras API for building and training deep learning models.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import json
import easyocr
from ultralytics import YOLO

### YOLO Model Initialization

Initialize YOLO model for pose estimation using pre-trained weights from Ultralytics library.


In [ ]:
model = YOLO("/kaggle/input/yolov8-pose-models/models/yolov8m-pose.pt" , verbose=False)

In [ ]:
def extract_jersey_from_frame(image_path):
    """
    Extracts the region containing the jersey number from a soccer player frame image.

    Args:
    - image_path (str): Path to the input image file.

    Returns:
    - img1 (numpy.ndarray): Cropped and resized image containing the jersey number region.
                            Returns None if jersey number extraction fails.
    """
    # Read the image
    image = cv2.imread(image_path)

    # Check if image is None
    if image is None:
        return None

    # Resize image to a fixed size
    image = cv2.resize(image, (285, 600))
    im_height, im_width = image.shape[:2]

    # Perform pose estimation using YOLO model
    results = model(image)
    keypoints = results[0].keypoints

    # Set confidence threshold for keypoints
    threshold = 0.1
    points = []

    # Extract keypoints with confidence above threshold
    for kpt_data in keypoints.data[0]:
        prob = kpt_data[2].item()
        if prob > threshold:
            x = kpt_data[0].item()
            y = kpt_data[1].item()
            points.append((int(x), int(y)))
        else:
            points.append(None)

    # Check if keypoints are visible
    if keypoints.has_visible is False:
        points = [None] * 13

    # Define quadrilateral coordinates for jersey region
    quadrilateral_coords = [points[6], points[5], points[12], points[11]]

    # Check if any of the points are None
    if None in quadrilateral_coords:
        return None

    # Calculate bounding box coordinates
    x1, y1 = points[5]
    x2, y2 = points[6]
    x3, y3 = points[11]
    x4, y4 = points[12]
    top = np.abs(min(y1, y2, y3, y4))
    left = np.abs(min(x1, x2, x3, x4))
    height = np.abs(max(y1, y2, y3, y4) - top)
    width = np.abs(max(x1, x2, x3, x4) - left)

    # Check if top or left coordinates are beyond image boundaries
    if top >= im_height * 0.5 or left >= im_width * 0.7:
        return None

    # Check if any coordinate is zero
    if 0 in [top, height, width, left]:
        return None

    # Crop and resize jersey region
    img1 = image[int(top + height * 0.1 - 3):int(top + height * 0.8 + 3), int(left - 10):int(left + width + 10)]
    try:
        img1 = cv2.resize(img1, (64, 32))
    except:
        return None

    # Check for invalid jersey region
    if x1 >= x2 or x1 >= x4 or x3 >= x4 or x3 >= x2 or y1 >= y3 or y1 >= y4 or y2 >= y3 or y2 >= y4 or np.abs(x2 - x1) < 55 or np.abs(x1 - x2) < 55:
        return None
    else:
        return img1

In [ ]:
reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory


In [ ]:
path = ["/kaggle/input/soccernet/Data/SoccerNet/jersey-2023/challenge/challenge/images/3/" + i for i in os.listdir("/kaggle/input/soccernet/Data/SoccerNet/jersey-2023/challenge/challenge/images/3")]

In [ ]:
label = []
for frame in path:
    jersey = extract_jersey_from_frame(frame)
    if jersey is not None :
#         new_image = cv2.resize(new_image, (64, 64))  
        gray_jersey = cv2.cvtColor(jersey ,  cv2.COLOR_BGR2GRAY)
        result = reader.readtext(gray_jersey, allowlist='0123456789')
        if len(result)>0:
            if result[0][-2]=='':
                label.append("-1")
            else:
                label.append(result[0][-2])
        else:
            label.append("-1")


In [ ]:
from collections import Counter

def calculate_percentage(lst):
    counts = Counter(lst)
    total_count = len(lst)
    percentages = {key: (count / total_count) * 100 for key, count in counts.items()}
    sorted_percentages = dict(sorted(percentages.items(), key=lambda item: item[1], reverse=True))
    return sorted_percentages
# Example usage:

percentage = calculate_percentage(label)
print("Percentage of each element:", percentage)

In [ ]:
def get_highest_percentages(percentage_dict):
    # Sort the dictionary by values in descending order
    sorted_percentage = sorted(percentage_dict.items(), key=lambda x: x[1], reverse=True)
    
    # Get the first two elements
    highest_percentages = sorted_percentage[:2]
    
    # Extract keys and values from the first two elements
    keys = [item[0] for item in highest_percentages]
    values = [item[1] for item in highest_percentages]
    
    return keys, values

# Example dictionary
# percentage = {'8': 10, '-1': 20, '6': 30, '18': 40, '75': 50, '0': 60, '1': 70}

# Call the function
keys, values = get_highest_percentages(percentage)

print("Keys of first two highest percentages:", keys)
print("Values of first two highest percentages:", values)


In [ ]:
from collections import Counter

def predict_majority_jersey_no_for_playerid(player_id, pred_label_list):
    """
    Predicts the majority jersey number for a player based on a list of labels.

    Args:
        player_id (int): The id number associated with the player.
        pred_label_list (list): A list of class labels.

    Returns:
        dict: A dictionary containing the player_id and the predicted label.
    """
    # Count occurrences of each label
    counts = Counter(pred_label_list)
    total_count = len(pred_label_list)
    
    # Calculate the percentage of each label
    percentages = {key: (count / total_count) * 100 for key, count in counts.items()}
    
    # Sort labels by percentage in descending order
    sorted_percentage = sorted(percentages.items(), key=lambda x: x[1], reverse=True)
    
    # Get the top two elements with the highest percentage
    top_two = sorted_percentage[:2]
    
    # Extract keys and values from the top two elements
    keys = [item[0] for item in top_two]
    values = [item[1] for item in top_two]
    
    # Check conditions to determine the predicted label
    if values[0] > 99.5 and keys[0] == "-1":
        pred_label = int(keys[0])  # If "-1" has a very high percentage, choose it
    elif int(values[0]) > int(values[1]) and keys[0] != "-1":
        pred_label = int(keys[0])  # Choose the label with the highest percentage if it's not "-1"
    else:
        pred_label = int(keys[1])  # Otherwise, choose the second label
    
    # Create a dictionary with the predicted label for the player
    pred_dict = {str(player_id): pred_label}
    
    return pred_dict


After defining the function for predicting the majority jersey number, I will apply this function to each player and store the results for further analysis.

In [ ]:
challenge_path = r"/kaggle/input/soccernet/Data/SoccerNet/jersey-2023/challenge/challenge/images"
predictions = {}

# Loop through player IDs
for player_id in range(1426):
    pred_label_list = []
    
    # Define the path to the folder containing player frames
    player_frame_folder_path = os.path.join(challenge_path, str(player_id))
    
    # Get paths to each frame for the player
    player_frames_path = [os.path.join(player_frame_folder_path, frame) for frame in os.listdir(player_frame_folder_path)]
    
    # Process each frame
    for frame_path in player_frames_path:
        print(frame_path)
        
        # Extract jersey from the frame
        jersey_extract_roi = extract_jersey_from_frame(frame_path)
        
        if jersey_extract_roi is not None:
            # Resize and convert the extracted jersey to grayscale
            resized_jersey_extract_roi = cv2.resize(jersey_extract_roi, (128, 128))  
            gray_jersey_extract_roi = cv2.cvtColor(resized_jersey_extract_roi, cv2.COLOR_BGR2GRAY)
            
            # Use EasyOCR to recognize text (jersey numbers) from the grayscale image
            predicted_output = reader.readtext(gray_jersey_extract_roi, allowlist='0123456789')
            
            # If text is recognized, append the predicted jersey number to the list
            if len(predicted_output) > 0:
                if predicted_output[0][-2] == '':
                    pred_label_list.append("-1")  # Append "-1" if no text is recognized
                else:
                    pred_label_list.append(predicted_output[0][-2])  # Append recognized jersey number
            else:
                pred_label_list.append("-1")  # Append "-1" if no text is recognized
#         else:
#             pred_label_list.append("-1")  # Append "-1" if no jersey is extracted
        pred_label_list.append("-1")
    # Predict the majority jersey number for the player
    predictions_dic = predict_majority_jersey_no_for_playerid(player_id, pred_label_list)
    
    # Add the prediction to the overall predictions dictionary
    predictions = {**predictions, **predictions_dic}

In [ ]:
print(predictions)

In [ ]:
file_path = 'output.json'

# Save dictionary to JSON file
with open(file_path, 'w') as json_file:
    json.dump(predictions, json_file)

print(f"JSON data saved to {file_path}")